In [2]:
%load_ext autoreload
%autoreload 2

# Setup


In [59]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
from gensim.models import KeyedVectors
from modules.nlp import NLP
from modules.visualisation import Visualisation

DATA_DIR = Path().cwd() / 'data'
EMBEDDINGS_DIR = Path().cwd() / 'embeddings'

nlp = NLP()
vis = Visualisation()

# Text preprocessing

In [47]:
posts_df = pd.read_csv(DATA_DIR / 'reddit_religion_combined.csv')

# Remove posts with no text
posts_df.dropna(subset='selftext', inplace=True)

# Tokenize text and remove stopwords
tqdm.pandas(desc='Tokenizing Reddit text')
posts_df['tokens'] = posts_df['selftext'].progress_apply(
    nlp.tokenize_text)

# Drop empty token cells
posts_df.dropna(subset=['tokens'], inplace=True)

Tokenizing Reddit text: 100%|██████████| 3127/3127 [00:01<00:00, 1795.33it/s]


# Embeddings


**Pretrained embeddings:** Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. Download from [this link](https://nlp.stanford.edu/data/glove.840B.300d.zip).


In [5]:
word_vectors = KeyedVectors.load_word2vec_format(
    EMBEDDINGS_DIR / 'glove.840B.300d.txt', binary=False, no_header=True)

**Generating word embeddings**

In [90]:
# Generate tf-idf matrix
tfidf_matrix, feature_names = nlp.generate_tfidf_matrix(
    posts_df['tokens'].values.tolist())

# Generate word embeddings
word_embeddings = nlp.generate_tfidf_weighted_embeddings(
    tokens=posts_df['tokens'].values.tolist(), tfidf_matrix=tfidf_matrix,
    feature_names=feature_names, word_vectors=word_vectors)
posts_df['weighted_embeddings'] = word_embeddings.tolist()

# Save processed data
posts_df.to_feather(DATA_DIR / 'reddit_religion_combined_embeddings.feather')

Generating weighted embeddings: 100%|██████████| 3077/3077 [00:07<00:00, 421.58it/s]


# Visualisation

**Plotting a 3D UMAP projection of the posts' embeddings**

In [ ]:
posts_df['dim_reduced_embeddings'] = vis.reduce_dimensionality(
    np.vstack(posts_df['weighted_embeddings'].values))

(3077,)